In [2]:
import numpy as np
import pandas as pd

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks

from sklearn.model_selection import train_test_split

2024-02-18 13:27:28.236075: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/phil/MultiNest/lib/:
2024-02-18 13:27:28.236146: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
# Import the MNIST data-set
# This data has alreay been converted to CSV and split into test and training sets, 
# to get more control over this we we will merge these sets then split them again later down the line
mnist_a= pd.read_csv('mnist/mnist_train.csv')
mnist_b = pd.read_csv('mnist/mnist_test.csv')
mnist = pd.concat([mnist_a, mnist_b])

In [4]:
# Quick look at the data
mnist.head()

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
# Separate labels and one-hot-encode them
X = mnist
y = X.pop('label')
y =pd.get_dummies(y)

y.head()

#Separate out training and validation data
X_train, X_val, y_train, y_val = \
    train_test_split(X, y, stratify=y, train_size=0.75)

In [6]:
# Set up the model
input_shape = [X_train.shape[1]]

model = keras.Sequential([
    layers.BatchNormalization(input_shape=input_shape),
    
    layers.Dense(units=256, activation='relu', input_shape=input_shape),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    
    layers.Dense(units=128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    
    layers.Dense(units=10, activation='softmax')
])

2024-02-18 13:27:50.408881: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/phil/MultiNest/lib/:
2024-02-18 13:27:50.408943: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2024-02-18 13:27:50.408985: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ULTIMA): /proc/driver/nvidia/version does not exist
2024-02-18 13:27:50.409492: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['acc'],
)

# Define early stopping callback
early_stopping = callbacks.EarlyStopping(
    min_delta = 0.0001,
    patience = 25,
    restore_best_weights = True
)

In [8]:
# Train the model
history = model.fit(
    X_train, y_train, 
    epochs=50, 
    batch_size=50,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping],
)

history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss', 'val_loss']].plot(title="Cross-entropy")
history_df.loc[:, ['acc', 'val_acc']].plot(title="Accuracy")

2024-02-18 13:28:12.164429: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/50
1050/1050 [==============================] - 7s 6ms/step - loss: 0.3364 - acc: 0.8966 - val_loss: 0.2571 - val_acc: 0.9543
Epoch 2/50
1050/1050 [==============================] - 6s 6ms/step - loss: 0.1759 - acc: 0.9458 - val_loss: 0.2234 - val_acc: 0.9632
Epoch 3/50
1050/1050 [==============================] - 6s 6ms/step - loss: 0.1375 - acc: 0.9563 - val_loss: 0.3239 - val_acc: 0.9675
Epoch 4/50
1050/1050 [==============================] - 6s 6ms/step - loss: 0.1175 - acc: 0.9629 - val_loss: 0.2330 - val_acc: 0.9710
Epoch 5/50
1050/1050 [==============================] - 6s 6ms/step - loss: 0.1054 - acc: 0.9664 - val_loss: 0.3854 - val_acc: 0.9708
Epoch 6/50
1050/1050 [==============================] - 6s 6ms/step - loss: 0.0937 - acc: 0.9706 - val_loss: 0.2726 - val_acc: 0.9728
Epoch 7/50
1050/1050 [==============================] - 6s 6ms/step - loss: 0.0831 - acc: 0.9730 - val_loss: 0.4126 - val_acc: 0.9738
Epoch 8/50
1050/1050 [==============================] - 6s 6ms

<AxesSubplot:title={'center':'Accuracy'}>